# Import data

In [2]:
import csv
data = []
for (counter,row) in enumerate(csv.reader(open("data/df-lowneg.csv"))):
    if counter == 0: continue
    data += [(counter,row[0],row[1])]
len(data),":",data[:5],"..",data[-5:]

(43968,
 ':',
 [(1,
   'DLLHGDESANSLDGGRGDDHLVGAEGDDWLIGGEGADRLEGGAGTDTADYSGAAARIVVGLNVNPLHGPEGGVGYEGEAHGDRLLDIENLVGSAHGDLLHGDDGANRIDGGKGDDVLVGAGGADTLDGGEGSDTASYAGSSARIEIDLLDGVGRGGEAEGDRLSGIENLLGTSHDDLLLGDAKANRIEGGDRGDTLNGRAGDDVLVGGEGGDWLDGGDGIDTADYSAAKGRVVVGLNVNTAWHADGGAGYEGE',
   'neg'),
  (2,
   'TADDSGVPTITWYQVGNGQPADIDAWYEQVNPYLEEKIGAHVNLQVVDWGSWNDRRTMLVQTNDDYDIIFTDMSTYASNVNMGAFADLTDLMANVPGLTDLIPEEYLKACNINGKLYGIPAYKDSSMTNFFVWTKDYVDQYYPDYAEDHDLFSIDEGLRAIYEGTGETPMMLNKDGISCIIGNRYDNFGSGLPAIGVSYYSDDAKVVSVFEQDDVLDQLRQMHTWYNDGLINSDANTLDNFQGMCAVGVAQGW',
   'neg'),
  (3,
   'GIYDAVPYALNLANVPFCGYGGEKDPQLLAATTMQQAAEKLQVPLKLIIGKGMGHAFDPASRNEFMAFHAANVQTGRPKFGTRDRIRFTTQTLKYNRCDWLSIIEMEQSGRPATVESERDAAGILQIRTDNVDLMTLARDITEQVRIDGTLLECRAAAGELLPEVWYRKTVDGWEVVDYQQSRTVSRNVDLNKRPGLQGPIDDAFMSPFICIRGTGQPLHPLPEAWSQRQLDVFQQEFARWMRGDVKVVNDVD',
   'neg'),
  (4,
   'EMIAIFLDLLGYRYDLTHDASDVDLGVKLAKQLVQHPLRNNFLHPLVFWLDRHSDITHDYSDLDTMLELARETVAGLAQNDPSRLDHLRNVAVIHYKRHRITNDLSDLDVCLKILLDVFEKTTTEPARHRIQGNIAKSLLRHYDI

In [ ]:
# ---------------
# example from https://github.com/facebookresearch/esm
#
# abandoned because doesn't run on my comp
#
#import torch
#model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t6_8M_UR50D")
#batch_converter = alphabet.get_batch_converter()
#model.eval()  # disables dropout for deterministic results
#batch_labels, batch_strs, batch_tokens = batch_converter(data)
#with torch.no_grad():
#    results = model(batch_tokens, repr_layers=[6], return_contacts=True)
#token_representations = results["representations"][6]
## ---------------------


# Model training, following https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/protein_language_modeling.ipynb to the T.

To use it, make sure you have installed `pip install transformers evaluate datasets requests pandas sklearn`

# Skip to next section if you just want to use the pre-trained model


In [3]:
model_checkpoint = "facebook/esm2_t6_8M_UR50D"

In [5]:
# split data
from sklearn.model_selection import train_test_split
transform_label = lambda x: {'neg':0,'rdrp':1,'xdxp':2}[x]
sequences, labels = [d[1] for d in data], [transform_label(d[2]) for d in data]
print(sequences[:5],labels[:5])
train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/esm2_t6_8M_UR50D")
print(tokenizer(train_sequences[0]))
train_tokenized = tokenizer(train_sequences)
test_tokenized = tokenizer(test_sequences)

['DLLHGDESANSLDGGRGDDHLVGAEGDDWLIGGEGADRLEGGAGTDTADYSGAAARIVVGLNVNPLHGPEGGVGYEGEAHGDRLLDIENLVGSAHGDLLHGDDGANRIDGGKGDDVLVGAGGADTLDGGEGSDTASYAGSSARIEIDLLDGVGRGGEAEGDRLSGIENLLGTSHDDLLLGDAKANRIEGGDRGDTLNGRAGDDVLVGGEGGDWLDGGDGIDTADYSAAKGRVVVGLNVNTAWHADGGAGYEGE', 'TADDSGVPTITWYQVGNGQPADIDAWYEQVNPYLEEKIGAHVNLQVVDWGSWNDRRTMLVQTNDDYDIIFTDMSTYASNVNMGAFADLTDLMANVPGLTDLIPEEYLKACNINGKLYGIPAYKDSSMTNFFVWTKDYVDQYYPDYAEDHDLFSIDEGLRAIYEGTGETPMMLNKDGISCIIGNRYDNFGSGLPAIGVSYYSDDAKVVSVFEQDDVLDQLRQMHTWYNDGLINSDANTLDNFQGMCAVGVAQGW', 'GIYDAVPYALNLANVPFCGYGGEKDPQLLAATTMQQAAEKLQVPLKLIIGKGMGHAFDPASRNEFMAFHAANVQTGRPKFGTRDRIRFTTQTLKYNRCDWLSIIEMEQSGRPATVESERDAAGILQIRTDNVDLMTLARDITEQVRIDGTLLECRAAAGELLPEVWYRKTVDGWEVVDYQQSRTVSRNVDLNKRPGLQGPIDDAFMSPFICIRGTGQPLHPLPEAWSQRQLDVFQQEFARWMRGDVKVVNDVD', 'EMIAIFLDLLGYRYDLTHDASDVDLGVKLAKQLVQHPLRNNFLHPLVFWLDRHSDITHDYSDLDTMLELARETVAGLAQNDPSRLDHLRNVAVIHYKRHRITNDLSDLDVCLKILLDVFEKTTTEPARHRIQGNIAKSLLRHYDITKDTSDLNRALQLVENVAINTVRQTVDTEYERPFQLTILAVCLCRRYIITRNCSDIDDALLWQQEAVNVTIKDDPQSKFRLG

In [10]:
from datasets import Dataset
train_dataset = Dataset.from_dict(train_tokenized)
test_dataset = Dataset.from_dict(test_tokenized)
print(train_dataset)
train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)
print(train_dataset)

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 32976
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 32976
})


In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = max(train_labels + test_labels) + 1  # Add 1 since 0 can be a label
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

#something to try some other time:
#from transformers import EsmTokenizer, EsmForSequenceClassification
#tokenizer = EsmTokenizer.from_pretrained(model_checkpoint)
#model = EsmForSequenceClassification.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at facebook/esm2_t6_8M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t6_8M_UR50D and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier

In [8]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 8

args = TrainingArguments(
    f"{model_name}-finetuned-localization",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)
from evaluate import load
import numpy as np

metric = load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [9]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

NameError: name 'train_dataset' is not defined

In [18]:
trainer.train()

/home/rayan/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 32976
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 12366
  Number of trainable parameters = 7840963
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: rayanc. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.147900,0.171905,0.958606
2,0.103500,0.146035,0.965975
3,0.075500,0.141601,0.969887


***** Running Evaluation *****
  Num examples = 10992
  Batch size = 8
Saving model checkpoint to esm2_t6_8M_UR50D-finetuned-localization/checkpoint-4122
Configuration saved in esm2_t6_8M_UR50D-finetuned-localization/checkpoint-4122/config.json
Model weights saved in esm2_t6_8M_UR50D-finetuned-localization/checkpoint-4122/pytorch_model.bin
tokenizer config file saved in esm2_t6_8M_UR50D-finetuned-localization/checkpoint-4122/tokenizer_config.json
Special tokens file saved in esm2_t6_8M_UR50D-finetuned-localization/checkpoint-4122/special_tokens_map.json
tokenizer config file saved in esm2_t6_8M_UR50D-finetuned-localization/tokenizer_config.json
Special tokens file saved in esm2_t6_8M_UR50D-finetuned-localization/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 10992
  Batch size = 8
Saving model checkpoint to esm2_t6_8M_UR50D-finetuned-localization/checkpoint-8244
Configuration saved in esm2_t6_8M_UR50D-finetuned-localization/checkpoint-8244/config.json
Model wei

TrainOutput(global_step=12366, training_loss=0.13043281034456958, metrics={'train_runtime': 1027.2949, 'train_samples_per_second': 96.3, 'train_steps_per_second': 12.037, 'total_flos': 1135514082546720.0, 'train_loss': 0.13043281034456958, 'epoch': 3.0})

In [20]:
trainer.save_model() 
trainer.save_state()

# Start here if you have the model saved locally!

In [8]:
# all you need to reload a trained model, mostly copypasted from above
import os
os.environ["WANDB_DISABLED"] = "true"
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained("esm2_t6_8M_UR50D-finetuned-localization", num_labels=num_labels)
model_checkpoint = "esm2_t6_8M_UR50D-finetuned-localization"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("esm2_t6_8M_UR50D-finetuned-localization")
model_name = model_checkpoint.split("/")[-1]
batch_size = 8
args = TrainingArguments(
    f"{model_name}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
)
from evaluate import load
import numpy as np
metric = load("accuracy")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
trainer = Trainer(
    model,
    args,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)
from datasets import Dataset

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Cloning https://huggingface.co/rchikhi/esm2_t6_8M_UR50D-finetuned-localization-finetuned-localization into local empty directory.


loading file vocab.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
/mnt/d/work/serratus/palmfold/esm2_t6_8M_UR50D-finetuned-localization-finetuned-localization is already a clone of https://huggingface.co/rchikhi/esm2_t6_8M_UR50D-finetuned-localization-finetuned-localization. Make sure you pull the latest changes with `repo.git_pull()`.


In [9]:
# test a prediction with a single protein

test_protein = "LAAKYRCRLRTIAPLVTFAFSLAVPILAAQQDPALPQPQQQSPQTTIRQTVRRVVVDVVVTDSHGKPVRGLTKDDFSVFEDGKPQKILSFDVHDSASDPAFVPPKLPALPANTFVNLPSGPERGPLYVLLYDLLNTEVEDQPQARKQLQKFIKSKPLGTRFAIFVLTDGLHLVQGFTTDRNLLLAAVDPGGSHIPRIFLYGDNFRPYVSGLGALIDIARFLSGLPGRKNLIWFSGSFPYFEDLPMLPGSDPSS"
test_tok  = tokenizer([test_protein])
test_ds = Dataset.from_dict(test_tok)
predictions = trainer.predict(test_ds)
print(np.argmax(predictions.predictions, axis=-1))
print(predictions)

***** Running Prediction *****
  Num examples = 1
  Batch size = 8


[0]
PredictionOutput(predictions=array([[ 5.986538 , -2.4479628, -2.8879347]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.98, 'test_samples_per_second': 1.02, 'test_steps_per_second': 1.02})


In [10]:
# predict wolf2018 (totally unseen given that training was serratusL)

prots = [] 
for line in open("data/RNAvirome.S2.fa"): #wolf2018
#for line in open("palmfold-git/pol/fa/all.fa"): # palmfold pol db
    if line.startswith('>'): continue
    prot = line.strip()[:1024]
    if len(prot) < 200: continue
    prots += [prot]
test_tok  = tokenizer(prots)
test_ds = Dataset.from_dict(test_tok)
predictions = trainer.predict(test_ds)
preds = np.argmax(predictions.predictions, axis=-1)
print(preds)
tp = 0
fp = 0
for p in preds:
    if p > 0:
        tp += 1
    else:
        fp += 1
print(tp,fp)
print("accuracy:",tp/(tp+fp))

# wolf2018:
# 4594 33
#accuracy: 0.9928679489950292

***** Running Prediction *****
  Num examples = 4627
  Batch size = 8


[2 2 2 ... 1 1 1]
4594 33
accuracy: 0.9928679489950292


In [11]:
# predict some other negs in serratusL unseen before

prots = [] 
for line in open("data/other-negs.csv"):
    prots += [line.split(',')[0]]
test_tok  = tokenizer(prots)
test_ds = Dataset.from_dict(test_tok)
predictions = trainer.predict(test_ds)
preds = np.argmax(predictions.predictions, axis=-1)
tp = 0
fp = 0
for p in preds:
    if p == 0:
        tp += 1
    else:
        fp += 1
print(tp,fp)
print("accuracy:",tp/(tp+fp))
# 9848 152
# accuracy: 0.9848

***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


9848 152
accuracy: 0.9848


In [13]:
# predict CFDL

prots = [] 
for line in open("data/CFDL-sample.fa"): 
    if line.startswith('>'): continue
    prot = line.strip()[:1024]
    if len(prot) < 200: continue
    prots += [prot]
test_tok  = tokenizer(prots)
test_ds = Dataset.from_dict(test_tok)
predictions = trainer.predict(test_ds)
preds = np.argmax(predictions.predictions, axis=-1)
tp = 0
fp = 0
for i,p in enumerate(preds):
    if p > 0:
        print(predictions.predictions[i],prots[i])

***** Running Prediction *****
  Num examples = 4032
  Batch size = 8


[ 0.6596159 -2.8863697  2.1669571] DTFYKLNIQEYRKERVAKGIDTNRAINEKDWFKESTRSNVQFNILREILRYKTSDSKCYSSMENNIFDILDQARTITNPILFSGSSELKKIFREARYKPEVANKYNISFIDGFGKDYICNIWYCIVYNSYFSDINFCILTTRELFEHIEFGKIMDDQFVKVSYKPSQSNNVGELSHNYWMQVTLAKDKPCIKTEIKNTDKSDDNQVLLSKATTL
[-1.7562011  2.303229  -0.4715117] MSFLEKEAEHVEGFAPELAVVTHGGGEKLEEPLVVRPTSETIIGYMYAKWIKSYRDLPVLINQWANVVRWEMRTRLFLRTLEFYWQEGHTAHATAEEAEEETRRMLDVYADFAENEAAVPVIPGRKSESEKFAGAVRTYSIEAMMGDGKALQSGTSHNLGQNFAKAFDIQYLDKNNELQYCWTTSWGLSTRFIGAIIMTHGDDQGLILPPRLAPIQVVIVPIYKNDEEKSKVMEVVERLRRELAGFRVKVDDR
[-2.479008  -2.4677284  4.441613 ] APAHARAPGLRSRAASPTLSIVAPINNSPVQGFRATWIQVNRDWFGRMRARLLERNCVGNIEVKRWRQRSLPDRQKSLSQPFVSAAIHQNGLSGDVRGALRSQPHDGIGDFARLAQALERSVRGPGVEDLLLGFSRRGGAGFGQFFQAIGGGETGADVVDQNSVFAELVGQALYQSHHRGTDGVGEHKIGDRLLRGNRGDGDDASPALALHVRDDFAGEVDRAQKVLLHGLAPFFEGGGEESLGWRAAGVGDT
[ 1.4464183  1.8378502 -2.9044936] ALNESPPTGGTSGTTNRRFTSYDRSPTTKLDYAVNRHYDPQQGRFTQVDPAGMKSTSLSNPQTLNLYAYCTNDPINRVDPSGLGFFSFLKKWFKRVWHAAIHAVFTFLQTLLMTGSVHAAFVAGVADFFKELGWPSK

In [42]:
# test accuracy on palmcores decoys

prots = [] 
labels = []
for line in open("data/palmcores.fa"): 
    if line.startswith('>'): 
        labels += [0 if 'decoy' in line else 1]
    else:
        prot = line.strip()[:1024]
        if len(prot) < 200: continue
        prots += [prot]
test_tok  = tokenizer(prots)
test_ds = Dataset.from_dict(test_tok)
#predictions = trainer.predict(test_ds)
#preds = np.argmax(predictions.predictions, axis=-1)
tp = 0
fp = 0
for i,p in enumerate(preds):
    if labels[i] == 0: #select only decoys
        if (labels[i] == 0 and (p == 0 or p == 2)) or \
            (labels[i] > 0 and p > 0):
                tp += 1
        else:
            fp +=1
        print(preds[i],labels[i])
print(tp,fp)
print("accuracy:",tp/(tp+fp))

1 0
0 0
2 0
2 0
1 0
0 0
2 0
0 0
0 0
0 0
1 0
2 0
0 0
0 0
0 0
0 0
2 0
2 0
2 0
0 0
2 0
2 0
1 0
0 0
2 0
2 0
0 0
2 0
2 0
2 0
2 0
0 0
2 0
0 0
0 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
0 0
0 0
0 0
0 0
0 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
2 0
